In [9]:
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
import numpy as np

# --- CONSTANTS ---
BASIC_EXPENDITURES = ['FOOD', 'CLOTH', 'HOUSING_WATER', 'HEALTH', 'EDUCATION']
MONTHLY_CAT_COLS = [f"{c}_MONTHLY" for c in BASIC_EXPENDITURES]

REGIONS = {
    1: "Region I - Ilocos Region", 2: "Region II - Cagayan Valley", 3: "Region III - Central Luzon",
    4: "Region IVA - CALABARZON", 5: "Region V - Bicol", 6: "Region VI - Western Visayas",
    7: "Region VII - Central Visayas", 8: "Region VIII - Eastern Visayas", 9: "Region IX - Zamboanga Peninsula",
    10: "Region X - Northern Mindanao", 11: "Region XI - Davao", 12: "Region XII - SOCCSKSARGEN",
    13: "National Capital Region", 14: "Cordillera Administrative Region",
    15: "Autonomous Region in Muslim Mindanao (ARMM)", 16: "Region XIII - Caraga",
    19: "Bangsamoro Autonomous Region in Muslim Mindanao", 17: "Region IVB - MIMAROPA",
}

REGION_MAP = {
    "Region I - Ilocos Region": "Region I (Ilocos Region)",
    "Region II - Cagayan Valley": "Region II (Cagayan Valley)",
    "Region III - Central Luzon": "Region III (Central Luzon)",
    "Region IVA - CALABARZON": "Region IV-A (CALABARZON)",
    "Region IVB - MIMAROPA": "MIMAROPA Region",
    "Region V - Bicol": "Region V (Bicol Region)",
    "Region VI - Western Visayas": "Region VI (Western Visayas)",
    "Region VII - Central Visayas": "Region VII (Central Visayas)",
    "Region VIII - Eastern Visayas": "Region VIII (Eastern Visayas)",
    "Region IX - Zamboanga Peninsula": "Region IX (Zamboanga Peninsula)",
    "Region X - Northern Mindanao": "Region X (Northern Mindanao)",
    "Region XI - Davao": "Region XI (Davao Region)",
    "Region XII - SOCCSKSARGEN": "Region XII (SOCCSKSARGEN)",
    "Region XIII - Caraga": "Region XIII (Caraga)",
    "National Capital Region": "National Capital Region (NCR)",
    "Cordillera Administrative Region": "Cordillera Administrative Region (CAR)",
    "Bangsamoro Autonomous Region in Muslim Mindanao": "Bangsamoro Autonomous Region In Muslim Mindanao"
}

# --- 1. DATA PROCESSING ---
def extract_basic_expenditures(filename: str) -> pd.DataFrame:
    # Load raw data
    fies = pd.read_csv(f'./datasets/raw/{filename}.csv')

    # Group and Mean (Annual)
    fies_grouped = fies[['W_REGN'] + BASIC_EXPENDITURES].groupby('W_REGN').mean()
    fies_grouped['REGION'] = fies_grouped.index.map(REGIONS)

    # Create Monthly Columns & Total
    for col in BASIC_EXPENDITURES:
        fies_grouped[f"{col}_MONTHLY"] = fies_grouped[col] / 12

    fies_grouped['TOTAL_MONTHLY'] = fies_grouped[MONTHLY_CAT_COLS].sum(axis=1)

    # Create "All Regions" Row (The National Average)
    nat_avg = fies_grouped.mean(numeric_only=True).to_frame().T
    nat_avg['REGION'] = "All Regions (National Avg)"

    # Combine
    return pd.concat([fies_grouped, nat_avg], ignore_index=True)

# Generate cleaned dataset
fies_2023 = extract_basic_expenditures('fies_2023')
fies_2023.to_csv('./datasets/clean/fies_2023.csv', index=False)

# --- 2. MAP & GEOMETRY PREP ---
ph_shp = gpd.read_file("./datasets/raw/Regions.shp.shp")
ph_shp['geometry'] = ph_shp['geometry'].simplify(0.01, preserve_topology=True)
ph_shp = ph_shp.to_crs(epsg=4326)

# Separate Map Data from National Avg row
regional_data = fies_2023[fies_2023['REGION'] != "All Regions (National Avg)"].copy()
regional_data['REGION_MAPPED'] = regional_data['REGION'].replace(REGION_MAP)

# Merge GeoData
ph_map = ph_shp.merge(regional_data, left_on='name', right_on='REGION_MAPPED').reset_index()
ph_json = json.loads(ph_map.to_json())

# Get reference to the National Average row for easy access
nat_avg_row = fies_2023[fies_2023['REGION'] == "All Regions (National Avg)"].iloc[0]

# --- 3. DASHBOARD CREATION ---
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    horizontal_spacing=0.15,
    specs=[[{"type": "mapbox"}, {"type": "xy"}]],
    subplot_titles=("Philippines Regions Map", "Expenditure Breakdown: National Average")
)

# Trace 0: Map
fig.add_trace(
    go.Choroplethmapbox(
        geojson=ph_json,
        locations=ph_map.index,
        z=ph_map['TOTAL_MONTHLY'],
        colorscale="YlOrRd",
        marker_opacity=0.7,
        marker_line_width=1,
        marker_line_color="black",
        hoverinfo='skip',
        colorbar=dict(title="Monthly PHP", x=0.53, thickness=15),
        selected={'marker': {'opacity': 1.0}},
        unselected={'marker': {'opacity': 0.3}}
    ), row=1, col=1
)

# Trace 1: Bar Chart (Initial state = National Avg)
fig.add_trace(
    go.Bar(
        x=BASIC_EXPENDITURES,
        y=nat_avg_row[MONTHLY_CAT_COLS].values,
        marker_color='#E65100',
        text=nat_avg_row[MONTHLY_CAT_COLS].values,
        texttemplate='₱%{text:,.0f}',
        textposition='outside',
        cliponaxis=False,
        hoverinfo='skip'
    ), row=1, col=2
)

# --- 4. INTERACTIVE MENUS ---
region_buttons = []

# Option: National Average
region_buttons.append(dict(
    label="All Regions (National Avg)",
    method="update",
    args=[
        {"selectedpoints": [list(range(len(ph_map))), None],
         "y": [None, nat_avg_row[MONTHLY_CAT_COLS].values],
         "text": [None, nat_avg_row[MONTHLY_CAT_COLS].values]},
        {"annotations[2].text": f"National Average Monthly Total: ₱{nat_avg_row['TOTAL_MONTHLY']:,.2f}",
         "annotations[1].text": "Expenditure Breakdown: National Average"}
    ]
))

# Options: Individual Regions
for i, row in ph_map.iterrows():
    region_buttons.append(dict(
        label=row['name'],
        method="update",
        args=[
            {"selectedpoints": [[i], None],
             "y": [None, row[MONTHLY_CAT_COLS].values],
             "text": [None, row[MONTHLY_CAT_COLS].values]},
            {"annotations[2].text": f"{row['name']} Monthly Total: ₱{row['TOTAL_MONTHLY']:,.2f}",
             "annotations[1].text": f"Expenditure Breakdown: {row['name']}"}
        ]
    ))

# Metric Selection
metric_buttons = []
metrics_to_show = ['TOTAL_MONTHLY'] + MONTHLY_CAT_COLS

for m in metrics_to_show:
    metric_buttons.append(dict(
        label=f"Map: {m.replace('_', ' ')}",
        method="restyle",
        args=[{"z": [ph_map[m]]}, [0]]
    ))

# --- 5. LAYOUT & STYLE ---
fig.update_layout(
    mapbox=dict(style="carto-positron", center={"lat": 12.8797, "lon": 121.7740}, zoom=4.5),
    dragmode=False,
    updatemenus=[
        dict(buttons=region_buttons, x=0.0, xanchor="left", y=1.35, yanchor="top"),
        dict(buttons=metric_buttons, x=0.6, xanchor="left", y=1.35, yanchor="top")
    ],
    height=800, margin=dict(t=200, b=50, l=50, r=50),
    yaxis2=dict(range=[0, ph_map[MONTHLY_CAT_COLS].max().max() * 1.2], fixedrange=True, tickformat="₱,.0f", side="right"),
    xaxis2=dict(fixedrange=True),
    showlegend=False
)

# Main Dashboard Annotation
fig.add_annotation(
    text=f"National Average Monthly Total: ₱{nat_avg_row['TOTAL_MONTHLY']:,.2f}",
    xref="paper", yref="paper", x=0.25, y=1.18,
    showarrow=False, font=dict(size=14, family="Arial Black"), xanchor="center"
)

fig.layout.annotations[0].update(y=1.05) # Philippines Map Title
fig.layout.annotations[1].update(y=1.05) # Bar Chart Title

# Initial State
fig.update_traces(selectedpoints=list(range(len(ph_map))), selector=dict(type="choroplethmapbox"))

config = {'displayModeBar': False, 'scrollZoom': False}
fig.write_html('./datasets/clean/interactive-dashboard.html', config=config)
fig.show(config=config)

/var/folders/q5/9qbnql8j3h31hh1lxczzrydh0000gn/T/ipykernel_48304/3015118869.py:95: DeprecationWarning:

*choroplethmapbox* is deprecated! Use *choroplethmap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

